In [10]:
%pip install yfinance pandas matplotlib plotly sqlalchemy mysql-connector-python PyMySQL 

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 779.6 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade nbformat
%pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install ipykernel
%pip install nbformat 



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime


/Users/darrielmarkerizal/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [17]:
# Symbol saham
symbol = "TLKM.JK"

# Tentukan rentang waktu 7 hari terakhir hingga hari ini
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = pd.to_datetime(end_date) - pd.DateOffset(days=7)

# Ambil data saham per jam menggunakan yfinance
tlkm_data = yf.download(symbol, start=start_date, end=end_date, interval='1h')

# Tambahkan kolom symbol dengan nilai yang sama pada setiap baris
tlkm_data['Symbol'] = symbol

print(tlkm_data)


[*********************100%%**********************]  1 of 1 completed
                             Open    High     Low   Close  Adj Close  \
Datetime                                                               
2023-12-29 09:00:00+07:00  3960.0  3960.0  3940.0  3960.0     3960.0   
2023-12-29 10:00:00+07:00  3950.0  3960.0  3950.0  3950.0     3950.0   
2023-12-29 11:00:00+07:00  3960.0  3960.0  3950.0  3950.0     3950.0   
2023-12-29 14:00:00+07:00  3950.0  3960.0  3940.0  3960.0     3960.0   
2023-12-29 15:00:00+07:00  3950.0  3980.0  3950.0  3970.0     3970.0   
2024-01-02 09:00:00+07:00  3950.0  3970.0  3940.0  3950.0     3950.0   
2024-01-02 10:00:00+07:00  3950.0  3970.0  3950.0  3960.0     3960.0   
2024-01-02 11:00:00+07:00  3960.0  3970.0  3950.0  3960.0     3960.0   
2024-01-02 13:00:00+07:00  3960.0  3970.0  3950.0  3970.0     3970.0   
2024-01-02 14:00:00+07:00  3960.0  3970.0  3960.0  3960.0     3960.0   
2024-01-02 15:00:00+07:00  3960.0  3970.0  3960.0  3970.0     3970.

In [6]:
# Membuat candlestick chart
fig = go.Figure(data=[go.Candlestick(x=tlkm_data.index,
                open=tlkm_data['Open'],
                high=tlkm_data['High'],
                low=tlkm_data['Low'],
                close=tlkm_data['Close'])])

# Menambahkan judul dan sumbu
fig.update_layout(
    title=f'Candlestick Chart - {symbol}',
    xaxis_title='Tanggal',
    yaxis_title='Harga',
    xaxis=dict(type='category')  # Tambahkan parameter ini
)

# Menampilkan chart
fig.show()


In [18]:
from sqlalchemy import create_engine

# Buat koneksi ke database MySQL
engine = create_engine('mysql+pymysql://root:@localhost/Finnovate')

# Dapatkan id untuk tipe 'stocks'
type_id = pd.read_sql_query("SELECT type_id FROM EquityType WHERE type_name = 'stocks'", engine).iloc[0, 0]

# Tambahkan kolom type_id dengan nilai yang sama pada setiap baris
tlkm_data['type_id'] = type_id

# Ubah indeks Datetime menjadi kolom biasa
tlkm_data.reset_index(inplace=True)

tlkm_data = tlkm_data.rename(columns={'level_0': 'symbol', 'index': 'symbol'})

# Ubah nama kolom agar sesuai dengan tabel Equity
tlkm_data.rename(columns={
    'Datetime': 'datetime',
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Adj Close': 'adj_close',
    'Volume': 'volume',
    'Symbol': 'symbol'
}, inplace=True)

# Masukkan data ke dalam tabel Equity
tlkm_data.to_sql('Equity', con=engine, if_exists='append', index=False)

23

In [19]:
# Query untuk mengambil data dari tabel Equity dengan symbol TLKM.JK
query = "SELECT datetime, open, high, low, close FROM Equity WHERE symbol = 'TLKM.JK'"

# Gunakan pandas untuk menjalankan query dan mendapatkan hasilnya sebagai DataFrame
df = pd.read_sql(query, con=engine)

# Buat candlestick chart dengan Plotly
fig = go.Figure(data=[go.Candlestick(x=df['datetime'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.show()

In [37]:
from sqlalchemy import create_engine, text
import pandas as pd

# Symbol saham
symbol = "TLKM.JK"

# Dapatkan objek Ticker
stock = yf.Ticker(symbol)

# Dapatkan data fundamental
fundamental_data = stock.info

# Inisialisasi variabel untuk menyimpan hasil looping
result_fundamental = ""

# Looping untuk mengumpulkan data fundamental
for key, value in fundamental_data.items():
    result_fundamental += f"{key}: {value}\n"

# Cetak hasil looping
print(result_fundamental)

# Ubah result_fundamental menjadi dictionary
result_dict = {}
for line in result_fundamental.split("\n"):
    if line:
        key, value = line.split(": ", 1)
        result_dict[key] = value

# Ubah dictionary menjadi DataFrame
df = pd.DataFrame([result_dict])

# Buat koneksi ke database MySQL
engine = create_engine('mysql+pymysql://root:@localhost/Finnovate')

# Hapus baris dengan underlyingSymbol yang sama
with engine.connect() as connection:
    connection.execute(text("DELETE FROM Company_Info WHERE underlyingSymbol = :symbol"), {'symbol': symbol})

# Masukkan data ke dalam tabel Company_Info
df.to_sql('Company_Info', con=engine, if_exists='append', index=False)

address1: Jl. Japati No. 1
city: Bandung
zip: 40133
country: Indonesia
phone: 62 22 452 7101
website: https://www.telkom.co.id
industry: Telecom Services
industryKey: telecom-services
industryDisp: Telecom Services
sector: Communication Services
sectorKey: communication-services
sectorDisp: Communication Services
longBusinessSummary: Perusahaan Perseroan (Persero) PT Telekomunikasi Indonesia Tbk provides information and communications technology, and telecommunications network services worldwide. The company operates through mobile, consumer, enterprise, Wholesale and International Business, and Other segments. The Mobile segment offers mobile voice, SMS, value added services, and mobile broadband services. The Consumer segment provides fixed wireline, pay TV, and internet services; and other telecommunication services to home customers. The Enterprise segment offers end-to-end solution to corporate and institutions. The Wholesale and International Business segment provides interconnec

1

In [75]:
# Symbol saham
symbol = "DOGE-USD"

# Tentukan rentang waktu 7 hari terakhir hingga hari ini
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = pd.to_datetime(end_date) - pd.DateOffset(days=7)

# Ambil data saham per jam menggunakan yfinance
btcusd_data = yf.download(symbol, start=start_date, end=end_date, interval='1h')

# Tambahkan kolom symbol dengan nilai yang sama pada setiap baris
btcusd_data['Symbol'] = symbol

print(btcusd_data)

[*********************100%%**********************]  1 of 1 completed
                               Open      High       Low     Close  Adj Close  \
Datetime                                                                       
2023-12-30 00:00:00+00:00  0.091031  0.091342  0.090958  0.091300   0.091300   
2023-12-30 01:00:00+00:00  0.091297  0.091401  0.091246  0.091330   0.091330   
2023-12-30 02:00:00+00:00  0.091332  0.091359  0.090662  0.090702   0.090702   
2023-12-30 03:00:00+00:00  0.090706  0.090941  0.090600  0.090759   0.090759   
2023-12-30 04:00:00+00:00  0.090757  0.090877  0.090484  0.090501   0.090501   
...                             ...       ...       ...       ...        ...   
2024-01-05 17:00:00+00:00  0.081909  0.081909  0.081235  0.081813   0.081813   
2024-01-05 18:00:00+00:00  0.081811  0.081924  0.081476  0.081791   0.081791   
2024-01-05 19:00:00+00:00  0.081805  0.082020  0.081673  0.081786   0.081786   
2024-01-05 20:00:00+00:00  0.081777  0.082434  0.08

In [46]:
# Dapatkan objek Ticker untuk BTC-USD
btc = yf.Ticker(symbol)

# Dapatkan informasi tentang BTC-USD
btc_info = btc.info

# Cetak informasi
for key, value in btc_info.items():
    print(f"{key}: {value}")

name: Bitcoin
startDate: 1278979200
description: Bitcoin (BTC) is a cryptocurrency launched in 2010. Users are able to generate BTC through the process of mining. Bitcoin has a current supply of 19,590,275. The last known price of Bitcoin is 44,148.68364498 USD and is up 2.69 over the last 24 hours. It is currently trading on 10693 active market(s) with $35,143,048,966.44 traded over the last 24 hours. More information can be found at https://bitcoin.org/.
maxAge: 86400
priceHint: 2
previousClose: 44178.953
open: 44178.953
dayLow: 44176.375
dayHigh: 44178.953
regularMarketPreviousClose: 44178.953
regularMarketOpen: 44178.953
regularMarketDayLow: 44176.375
regularMarketDayHigh: 44178.953
volume: 32224952320
regularMarketVolume: 32224952320
averageVolume: 20217814264
averageVolume10days: 27124399814
averageDailyVolume10Day: 27124399814
marketCap: 865450786816
fiftyTwoWeekLow: 16716.422
fiftyTwoWeekHigh: 45899.707
fiftyDayAverage: 41069.926
twoHundredDayAverage: 32293.188
currency: USD
fr

In [56]:
# Symbol saham
symbol = "ETH-USD"

# Dapatkan objek Ticker
btc = yf.Ticker(symbol)

# Dapatkan data fundamental
fundamental_data = btc.info

# Ubah dictionary menjadi DataFrame
df = pd.DataFrame([fundamental_data])

# Hapus baris dengan underlyingSymbol yang sama
with engine.connect() as connection:
    connection.execute(text("DELETE FROM Crypto_Info WHERE symbol = :symbol"), {'symbol': symbol})

# Masukkan data ke dalam tabel BitcoinInfo
df.to_sql('Crypto_Info', con=engine, if_exists='append', index=False)

1

In [74]:
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine, text
from datetime import datetime

# Buat koneksi ke database
engine = create_engine('mysql+pymysql://root:@localhost/Finnovate')

# Symbol crypto
symbol = "DOGE-USD"

# Dapatkan objek Ticker
eth = yf.Ticker(symbol)

# Dapatkan data fundamental
fundamental_data = eth.info

# Ubah dictionary menjadi DataFrame
df = pd.DataFrame([fundamental_data])

# Cek apakah simbol sudah ada di tabel Crypto_Info
with engine.connect() as connection:
    result = connection.execute(text("SELECT id FROM Crypto_Info WHERE symbol = :symbol"), {'symbol': symbol})

crypto_id = result.scalar()  # Mengambil id jika ada, jika tidak ada akan mengembalikan None

# Jika simbol belum ada di tabel Crypto_Info, tambahkan ke tabel dan ambil id yang baru dibuat
if crypto_id is None:
    df.to_sql('Crypto_Info', con=engine, if_exists='append', index=False)
    with engine.connect() as connection:
        crypto_id = connection.execute(text("SELECT id FROM Crypto_Info WHERE symbol = :symbol"), {'symbol': symbol}).scalar()

# Tentukan rentang waktu 7 hari terakhir hingga hari ini
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = pd.to_datetime(end_date) - pd.DateOffset(days=7)

# Ambil data crypto per jam menggunakan yfinance
history_data = yf.download(symbol, start=start_date, end=end_date, interval='1h')

# Tambahkan kolom crypto_id dan symbol
history_data['crypto_id'] = crypto_id
history_data['symbol'] = symbol

# Dapatkan id untuk tipe 'crypto'
type_id = pd.read_sql_query("SELECT type_id FROM EquityType WHERE type_name = 'crypto'", engine).iloc[0, 0]

# Tambahkan kolom type_id dengan nilai yang sama pada setiap baris
history_data['type_id'] = type_id

# Ubah indeks Datetime menjadi kolom biasa
history_data.reset_index(inplace=True)

# Ubah nama kolom agar sesuai dengan tabel Equity
history_data = history_data.rename(columns={'Datetime': 'datetime', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Adj Close': 'adj_close', 'Volume': 'volume'})

# Masukkan data ke dalam tabel Equity
history_data.to_sql('Equity', con=engine, if_exists='append', index=False)

[*********************100%%**********************]  1 of 1 completed


166